In [2]:
from ptycho_v2.multires_v2.class_multiressolver import *
import matplotlib.pyplot as plt
import torch
from ptycho_v2.tools_v2.ptychography import Ptychography as Ptychography
from utils import *

In [3]:
def run_test():
    # Setting the operating device as cpu and the inital as ...
    max_scale = 7
    max_probe_size = 32
    max_shift = 8
    device = 'cuda'
    I_in = 15*np.array([1, 15, 10, 5, 10, 30, 100])
    I_out = 20*np.array([0, 0, 0, 7, 6, 5, 80])
    #I_out = 300*np.array([0, 0, 0, 0, 0, 0, 0, 0, 10])
    cycle = [0, -1, -1, -1,  1, 1, 1]
    lmbda = 0
    LR = 1e-2
    tol = [1e-10] * 9
    tol_in = [1e-10] * 9

    linOperator = Ptychography(max_scale = max_scale,max_probe_size = max_probe_size ,max_shift = max_shift,device=device)

    image = plt.imread('images/peppers_reduced.jpg')/ 255
    image_tensor = torch.tensor(image).double().to(device).view(1, 1, 2**max_scale, 2**max_scale)
    image_tensor_ = torch.exp(1j * image_tensor)
    #Initiate the MultiRes class with the inital scale.
    multires = MultiRes(max_scale, device)
    #loss = Loss(linOperator,linOperator.apply(image_tensor_), lmbda = lmbda)
    loss = Loss(linOperator,linOperator.apply(image_tensor_))
    model = MultiResSolver(multires, loss, LR = LR,
                           I_in = I_in,
                           I_out = I_out,
                           tol = tol,
                           tol_in = tol_in,
                           cycle = cycle,
                           l1_type = "l1_row")
    
    model.solve_multigrid()
    model.print_time()
    return model

In [4]:
image = plt.imread('images/peppers_reduced.jpg')/255
model = run_test()

----------- s = 7 -----------
----------- s = 6 -----------
----------- s = 5 -----------
----------- s = 4 -----------
Iter 1, [loss, mse, reg, rel_loss, LR] : [5616.9127978, 5616.9127978, 4.65e-05, 0.01] 
Iter 2, [loss, mse, reg, rel_loss, LR] : [5616.6518483, 5616.6518483, 4.71e-05, 0.01] 
Iter 3, [loss, mse, reg, rel_loss, LR] : [5616.3874574, 5616.3874574, 6.11e-05, 0.01] 
Iter 4, [loss, mse, reg, rel_loss, LR] : [5616.0442779, 5616.0442779, 7.54e-05, 0.01] 
Iter 5, [loss, mse, reg, rel_loss, LR] : [5615.6210244, 5615.6210244, 9.01e-05, 0.01] 
Iter 6, [loss, mse, reg, rel_loss, LR] : [5615.1149914, 5615.1149914, 0.0001055, 0.01] 
Iter 7, [loss, mse, reg, rel_loss, LR] : [5614.5223666, 5614.5223666, 0.0001218, 0.01] 
Iter 8, [loss, mse, reg, rel_loss, LR] : [5613.8383271, 5613.8383271, 0.0001392, 0.01] 
Iter 9, [loss, mse, reg, rel_loss, LR] : [5613.0570474, 5613.0570474, 0.0001577, 0.01] 
Iter 10, [loss, mse, reg, rel_loss, LR] : [5612.1716619, 5612.1716619, 0.0001777, 0.01] 
Iter

KeyboardInterrupt: 

In [ ]:
plot_results(model,image)